
## Traitement de flux structuré avec l'API DataFrame Python 
Apache Spark inclut une API de traitement de flux de haut niveau : Structured Streaming.
<br>Dans ce TP, nous jetons un coup d'œil rapide l'utilisation de l'API DataFrame pour construire des applications Structured Streaming.
<br>Dans un premier temps, nous réviserons les éléments vus précédemment pour explorer rapidement les données.
<br>Dans un second temps, nous calculerons des métriques en temps réel (dans notre exemple, ce seront des "running counts" et "windowed counts" sur un flux d'actions horodatées).


Uploader les 50 fichiers du dossier events dans le dossier /FileStore/tables/events. Vérifier qu'ils sont bien présents :

In [ ]:
# Votre code ici

Exécuter la commande suivante pour voir rapidement ce que les fichiers contiennent à partir de l'exemple du fichier "file_0.json".

In [ ]:
%fs head /FileStore/tables/events/file_0.json

Chaque ligne du fichier contient un enregistrement json avec deux champs : `time` and `action`. Nous allons analyser ces fichiers de manière interactive.

## Traitement par lots/interactif
La première étape pour tenter de traiter les données est de les interroger de manière interactive.
Nous allons définir un DataFrame statique sur les fichiers.

In [ ]:
input_path = "/FileStore/tables/events"

Pour changer (et en pratique accélérer le processus), définir le schéma (au lieu de demander à Spark de l'inférer).

In [ ]:
from pyspark.sql.types import # à compléter
# Votre code ici

Calculer le nombre d'actions "open" et "close" dans des fenêtres d'une heure.
<br>Indication : utiliser un GROUP BY sur la colonne `action` et des fenêtres (windows) sur la colonne `time`.
<br>Réaliser cette tâche en DSL.
<br>Mettre en cache (en mémoire) le DataFrame obtenu.

In [ ]:
from pyspark.sql.functions import window

# Votre code ici

<br>Déterminer le nombre total d'actions "open" et d'actions "close" (sur l'ensemble de la période).
<br>Réaliser cette tâche en SQL.

In [ ]:
# Votre code ici

Dans un notebook Databricks, il est possible d'exécuter une commande sql en ligne de commande avec la commande %sql, et d'obtenir un visuel automatique.
<br> L'essayer sur l'instruction précédente.

In [ ]:
# Votre code ici

L'utilisation des windows se fait de la manière suivante en SQL. Exécuter la commande suivante pour obtenir un visuel des actions dans le temps.

In [ ]:
%sql select action, date_format(window.end, "MMM-dd HH:mm") as time, count from static_counts order by time, action

Vous devriez remarquer que les actions de fermeture suivent les actions d'ouverture correspondantes (il y a plus d'"open" au début et plus de "close" à la fin).

### Traitement de flux (stream processing)
Maintenant que nous avons analysé les données de manière interactive, nous allons créer une requête de traitement de flux qui se met à jour en continu, à mesure que les données arrivent.
<br>Étant donné que nous avons simplement un ensemble statique de fichiers, nous allons émuler un flux à partir d'eux en lisant un fichier à la fois, dans l'ordre chronologique de leur création.
<br> A noter que le code que nous allons écrire est à peu près le même que précédemment. Merci l'API de Spark !

Adapter le code créé précédemment pour calculer le nombre d'actions "open" et "close", en utilisant readStream à la place de read. Utiliser l'option maxFilesPerTrigger afin de spécifier qu'on ne prend qu'un ficher à la fois (les options doivent être placées avant le chargement effectif à partir de la méthode json).

In [ ]:
# Votre code ici

En fouillant (avec le nez fin) la documentation des DataFrames (https://spark.apache.org/docs/3.1.3/api/python/reference/api/pyspark.sql.DataFrame.html?highlight=dataframe#pyspark.sql.DataFrame), vérifier si le DataFrame que nous venons de créer est bien un DataFrame de streaming.

In [ ]:
# Votre code ici

<br>Vous pouvez démarrer le calcul en continu en définissant la destination (le "sink") et en le démarrant.
<br>Dans notre cas, nous voulons interroger de manière interactive les comptages (mêmes requêtes que ci-dessus), nous allons donc définir l'ensemble complet des comptages d'une heure dans une table en mémoire (pour cela il faudra utiliser format avec l'option "memory").
<br>Utiliser la fonction writeStream pour démarrer la requête.
<br>Donner un nom à la table in-memory, afin de pouvoir y faire référence dans la suite. Pour cela, utiliser queryName.
<br>Choisir un outputMode "complete".

In [ ]:
# Votre code ici

`query` est une référence à la requête de streaming qui s'exécute en arrière-plan. Cette requête récupère continuellement les fichiers et met à jour les comptages fenêtrés. A noter que nous ne faisons que simuler une arrivée continue de fichiers, grâce à l'option maxFilesPerTrigger. Nous n'avons pas précisé d'intervalle de temps entre chaque trigger (il est possible de le paramétrer).

Remarquer le statut de la requête dans la cellule ci-dessus (la barre de progression montre que la requête est active).
<br> Développer `> counts` : on y trouve le nombre de fichiers traités.

Attendre un peu (physiquement ou en utilisant la fonction sleep de la bibliothèque time) puis lancer la requête sql permettant de visualiser les comptages dans le temps sur le DataFrame de streaming. Ne pas oublier que nous avons donné un nom particulier à la table in-memory.

In [ ]:
# Votre code ici

Continuer à exécuter la requête de manière itérative.

In [ ]:
# Votre code- (éventuel) ici

Visualiser le nombre total d'actions "open" et "close".

In [ ]:
# Votre code ici

Si vous exécutez la requête ci-dessus de manière répétée, vous constaterez que le nombre d'actions "open" est bien en permanence plus élevé que le nombre d'actions "close", comme anticipable dans un flux de données où une fermeture apparaît toujours après l'ouverture qui lui correspondant. Cela montre que Structured Streaming garantit l'"intégrité du préfixe" (prefix integrity).

Arrêter la requête en exécutant query.stop(). Vérifier le statut de la cellule correspondante à la requête.
<br>Ici bien entendu, nous n'avons que quelques fichiers, et il est possible qu'au moment où vous arrivez à cette cellule, ils soient tous consommés. Dans ce cas, il n'y a plus de mise à jour des comptages. Mais il faut quand même arrêter notre requête proprement !

In [ ]:
# Votre code ici